# 🧹 SAP Report Cleaner

**Bereinigt SAP-Reports in 3 einfachen Schritten!**

---

## So geht's:

1. **▶️ Klicken** auf den Play-Button bei der nächsten Zelle
2. **📂 Datei auswählen** - Ein Dialog öffnet sich, wählen Sie Ihre SAP-Datei
3. **💾 Herunterladen** - Die bereinigte Datei landet in Ihrem Downloads-Ordner

**Fertig!** Keine Installation, kein Pfad eingeben, einfach klicken.

---


In [ ]:
#@title 🚀 **Ausführen - Klicken Sie auf ▶️ links** { display-mode: "form" }
#@markdown ---
#@markdown ### Dieses Script:
#@markdown - Verbindet mit Google Drive
#@markdown - Lässt Sie eine SAP-Datei auswählen
#@markdown - Bereinigt die Daten automatisch
#@markdown - Speichert die Ergebnisse in Ihrem Drive
#@markdown ---

# ============================================================
# IMPORTS UND SETUP
# ============================================================

import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import os
import io

# Google Colab spezifische Imports
from google.colab import files
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# ============================================================
# KONFIGURATION
# ============================================================

EXPECTED_HEADERS = [
    'Material', 'Functional Loc.', 'Equipment', 'Material Description',
    'Work Ctr', 'Withdrawn', 'W/o resrv.', 'Reserved', 'Reserv.ref',
    'Pstng Date', 'Order', 'ID', 'Message', 'ICt', 'Customer'
]

TEXT_COLUMNS = ['Functional Loc.', 'Equipment', 'Material Description',
                'Work Ctr', 'ID', 'ICt', 'Customer']
DATE_COLUMN = 'Pstng Date'
NUMERIC_COLUMNS = ['Material', 'Withdrawn', 'W/o resrv.', 'Reserved',
                   'Reserv.ref', 'Order', 'Message']

# ============================================================
# HILFSFUNKTIONEN
# ============================================================

def clean_number(value):
    """Bereinigt Zahlenwerte aus SAP-Format."""
    if pd.isna(value) or value is None:
        return None
    val_str = str(value).strip()
    if val_str == '' or val_str == '-':
        return None
    val_str = val_str.replace('\xa0', '').replace(' ', '')
    if ',' in val_str and '.' in val_str:
        val_str = val_str.replace('.', '').replace(',', '.')
    elif ',' in val_str:
        parts = val_str.split(',')
        if len(parts) == 2 and len(parts[1]) <= 2:
            val_str = val_str.replace(',', '.')
        else:
            val_str = val_str.replace(',', '')
    elif '.' in val_str:
        parts = val_str.split('.')
        if len(parts) == 2 and len(parts[1]) == 3 and len(parts[0]) >= 1:
            val_str = val_str.replace('.', '')
    try:
        return int(round(float(val_str)))
    except ValueError:
        return None

def convert_date(value):
    """Konvertiert Datum aus SAP-Format."""
    if pd.isna(value) or value is None:
        return ''
    val_str = str(value).strip()
    if val_str == '':
        return ''
    for fmt in ['%d.%m.%y', '%d.%m.%Y', '%Y-%m-%d']:
        try:
            return datetime.strptime(val_str, fmt).strftime('%d.%m.%Y')
        except ValueError:
            continue
    return val_str

def process_sap_report(content):
    """Verarbeitet SAP-Report Inhalt."""
    lines = content.split('\n')
    all_rows = [line.split('\t') for line in lines]

    # Header finden
    header_row_idx, header_start_col = None, None
    for idx, row in enumerate(all_rows):
        for col_idx, cell in enumerate(row):
            if str(cell).strip().lower() == 'material':
                header_row_idx, header_start_col = idx, col_idx
                break
        if header_row_idx is not None:
            break

    if header_row_idx is None:
        header_row_idx, header_start_col = 3, 2

    # Daten verarbeiten
    cleaned_data, deleted_rows = [], []
    stats = {'total': 0, 'sum_rows': 0, 'empty': 0, 'no_material': 0, 'kept': 0}

    for row_idx in range(header_row_idx + 1, len(all_rows)):
        row = all_rows[row_idx]
        stats['total'] += 1

        if all(str(cell).strip() == '' for cell in row):
            stats['empty'] += 1
            continue

        col_b = str(row[1]).strip() if len(row) > 1 else ''
        if col_b in ['*', '**']:
            stats['sum_rows'] += 1
            deleted_rows.append({'Grund': 'Summenzeile', 'Zeile': row_idx + 1,
                                 'Daten': '\t'.join(str(c) for c in row)})
            continue

        data_row = [str(row[i]).strip() if i < len(row) else ''
                    for i in range(header_start_col, header_start_col + 15)]

        if not data_row[0]:
            stats['no_material'] += 1
            deleted_rows.append({'Grund': 'Keine Materialnummer', 'Zeile': row_idx + 1,
                                 'Daten': '\t'.join(data_row)})
            continue

        cleaned_data.append(data_row)
        stats['kept'] += 1

    # DataFrame erstellen
    df = pd.DataFrame(cleaned_data, columns=EXPECTED_HEADERS)
    df_deleted = pd.DataFrame(deleted_rows)

    # Datentypen konvertieren
    for col in NUMERIC_COLUMNS:
        if col in df.columns:
            df[col] = df[col].apply(clean_number)
    if DATE_COLUMN in df.columns:
        df[DATE_COLUMN] = df[DATE_COLUMN].apply(convert_date)

    return df, df_deleted, stats

print("✅ Bereit!")

# ============================================================
# EINFACHE OBERFLÄCHE - NUR 3 SCHRITTE
# ============================================================

# Globale Variablen
result_df = None
result_deleted = None
result_stats = None
source_filename = None

# UI Elemente
output_area = widgets.Output()

format_dropdown = widgets.Dropdown(
    options=['📊 Excel (.xlsx)', '📄 CSV (.csv)'],
    value='📊 Excel (.xlsx)',
    description='',
    layout=widgets.Layout(width='200px')
)

upload_btn = widgets.Button(
    description='1️⃣ SAP-Datei auswählen',
    button_style='primary',
    layout=widgets.Layout(width='250px', height='50px'),
    style={'font_weight': 'bold'}
)

process_btn = widgets.Button(
    description='3️⃣ Bereinigen & Herunterladen',
    button_style='success',
    layout=widgets.Layout(width='280px', height='50px'),
    style={'font_weight': 'bold'}
)

status_label = widgets.HTML(value='<span style="font-size:16px">⏳ Warte auf Datei...</span>')

def on_upload_click(b):
    global result_df, result_deleted, result_stats, source_filename
    with output_area:
        clear_output()
        print("📤 Bitte Datei aus Ihrem Downloads-Ordner auswählen...")
        print("   (Ein Datei-Dialog öffnet sich gleich)\n")
        
        try:
            uploaded = files.upload()
        except Exception as e:
            print(f"❌ Fehler beim Upload: {e}")
            return
            
        if not uploaded:
            print("❌ Keine Datei ausgewählt")
            status_label.value = '<span style="color:red;font-size:16px">❌ Keine Datei</span>'
            return
            
        source_filename = list(uploaded.keys())[0]
        content = uploaded[source_filename].decode('utf-8', errors='replace')

        print(f"📄 Datei: {source_filename}")
        print("⏳ Verarbeite...")
        
        result_df, result_deleted, result_stats = process_sap_report(content)
        
        print(f"\n✅ Fertig geladen!")
        print(f"\n📊 Ergebnis:")
        print(f"   ✓ {result_stats['kept']} bereinigte Zeilen")
        print(f"   ✗ {result_stats['sum_rows']} Summenzeilen entfernt")
        print(f"   ✗ {result_stats['no_material']} ohne Materialnr. entfernt")
        print(f"\n📋 Vorschau:")
        display(result_df.head())
        
        status_label.value = f'<span style="color:green;font-size:16px">✅ {result_stats["kept"]} Zeilen bereit - Jetzt Schritt 3!</span>'

def on_process_click(b):
    global result_df, result_deleted, source_filename
    with output_area:
        if result_df is None:
            print("\n❌ Bitte zuerst Schritt 1: Datei auswählen!")
            status_label.value = '<span style="color:red;font-size:16px">❌ Erst Datei laden!</span>'
            return
            
        base_name = Path(source_filename).stem
        
        if '📊' in format_dropdown.value:
            output_filename = f"{base_name}_cleaned.xlsx"
            output_path = f"/content/{output_filename}"
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                result_df.to_excel(writer, sheet_name='Bereinigte Daten', index=False)
                if not result_deleted.empty:
                    result_deleted.to_excel(writer, sheet_name='Gelöschte Zeilen', index=False)
        else:
            output_filename = f"{base_name}_cleaned.csv"
            output_path = f"/content/{output_filename}"
            result_df.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
        
        print(f"\n📥 Download startet: {output_filename}")
        print(f"   → Datei erscheint in Ihrem Downloads-Ordner!")
        
        files.download(output_path)
        
        status_label.value = f'<span style="color:green;font-size:16px">✅ Download: {output_filename}</span>'

upload_btn.on_click(on_upload_click)
process_btn.on_click(on_process_click)

# Layout anzeigen
display(widgets.HTML('''
<div style="background:#f0f7ff; padding:20px; border-radius:10px; margin:10px 0;">
<h2 style="margin:0 0 15px 0;">🧹 SAP Report Cleaner</h2>
<p style="margin:0; color:#666;">Bereinigt SAP-Reports in 3 einfachen Schritten</p>
</div>
'''))

display(widgets.HTML('<h3>Schritt 1: Datei hochladen</h3>'))
display(widgets.HTML('<p>Klicken Sie den Button → Wählen Sie Ihre SAP-Datei aus dem Downloads-Ordner</p>'))
display(upload_btn)

display(widgets.HTML('<h3>Schritt 2: Format wählen</h3>'))
display(format_dropdown)

display(widgets.HTML('<h3>Schritt 3: Herunterladen</h3>'))
display(widgets.HTML('<p>Die bereinigte Datei wird in Ihren <b>Downloads-Ordner</b> heruntergeladen</p>'))
display(process_btn)

display(widgets.HTML('<hr>'))
display(status_label)
display(output_area)


---

## 💡 Hinweise

### Wo ist meine bereinigte Datei?
Nach dem Klick auf "Herunterladen" wird die Datei automatisch in Ihren **Downloads-Ordner** gespeichert - genau wie jeder andere Browser-Download!

### Excel vs CSV
| Format | Inhalt |
|--------|--------|
| **Excel** | 2 Tabellenblätter: Bereinigte Daten + Gelöschte Zeilen |
| **CSV** | Nur die bereinigten Daten |

---

## ❓ Probleme?

| Problem | Lösung |
|---------|--------|
| Download startet nicht | Popup-Blocker für colab.google.com deaktivieren |
| Keine Daten | Prüfen ob die SAP-Datei Tab-getrennt ist (.txt) |
| Datei-Dialog öffnet sich nicht | Seite neu laden, nochmal versuchen |

---

*SAP Report Cleaner v1.2 - Januar 2026*
